In [ ]:
import numpy as np
from imripy import merger_system as ms
from imripy import halo
from imripy import inspiral
from imripy import waveform
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
# Basic system and spike properties
g_spike = 7./3.
m1 = 1e5*ms.solar_mass_to_pc
m2 = 1e3*ms.solar_mass_to_pc
#rho_spike = 226.*ms.solar_mass_to_pc
#r_spike = ( (3 - g_spike) * m1 / (2 * np.pi * rho_spike) * 0.2**(3.-g_spike) )**(1./3.)
D = 5e8
model = "Form.m5.m3.alpha2.3"

In [ ]:
# Model the system with spike
potential = lambda r: m1/r
Eps_grid = np.geomspace(1e-13, 1e1, 1000)

NFWSpike = halo.Spike.FromSpikedNFW(halo.SpikedNFW.FromNFW(halo.NFW.FromHaloMass(1e3*m1, 20.), 
                                                            m1, g_spike))
dh = halo.DynamicSS.FromStatic(Eps_grid, NFWSpike, potential)

# This is so that we can extend the grid easily for the evolution
f_grid_interp = interp1d(dh.Eps_grid, dh.f_grid, kind='cubic', fill_value=(0.,0.), bounds_error=False, copy=True)

sp = ms.SystemProp(m1, m2, dh, D)

In [ ]:
# Model the inspiral
R0 = 60.* sp.r_isco()
R_fin = 1. * sp.r_isco()
r_grid = np.geomspace(sp.r_isco(), 50*R0, 1000)

Eps_grid = np.geomspace(1e-13, 1e1, 1000)
Eps_grid = np.sort(np.append(Eps_grid, np.geomspace(1e-1 * (sp.m1/R0 - (sp.omega_s_approx(R0)*R0)**2 / 2.), 1e1 * sp.m1/R0, 2000)))

sp.halo.Eps_grid = Eps_grid; sp.halo.update_Eps()
sp.halo.f_grid = f_grid_interp(Eps_grid)
haloModel = inspiral.HaloModel(sp)

In [ ]:
# Plot initial configuration
fig, (ax_rho, ax_f) = plt.subplots(2, 1, figsize=(20,20))
ax_rho.loglog(r_grid, NFWSpike.density(r_grid), label='analytic')
ax_rho.loglog(r_grid, dh.density(r_grid), linestyle='--', label=r'recovered')
ax_rho.axvline(R0, linestyle = '--', color='black', label = r'$R_0$')
ax_rho.set_xlabel('r / pc')
ax_rho.legend();
ax_f.loglog(dh.Eps_grid, dh.f_grid, label="$f$")
ax_f.axvline(sp.m1/R0, linestyle='-.', label='$m1/r$')
ax_rho.grid(); ax_f.grid()
plt.legend(); 

In [ ]:
# Evolve the system
t, R, f = haloModel.evolve_circular_binary( R0, R_fin = R_fin, acc=1e-8)

In [ ]:
from matplotlib.animation import FuncAnimation
n_frame = len(t)-1
#n_frame = 10

print(n_frame)
fig, (ax_rho, ax_f) = plt.subplots(2, 1, figsize=(20,20))

index = 0;
v_0 = sp.omega_s_approx(R0)*R0
Tini_orb =  2.*np.pi / sp.omega_s_approx(R0)
lr0 = ax_rho.axvline(R0/sp.r_isco(), linestyle='-.', label='$r_0$', color='black')

dh.f_grid = f[0,:]
lrho, = ax_rho.loglog(r_grid/sp.r_isco(), dh.density(r_grid), label=r'$\rho$')
lrho_v0, = ax_rho.loglog(r_grid/sp.r_isco(), dh.density(r_grid, v_max=[sp.omega_s_approx(r)*r for r in r_grid]), 
                                  color=lrho.get_c(), linestyle='--', label=r'$\rho_{v<v_{orb}}$')

lr = ax_rho.axvline(R[0]/sp.r_isco(), linestyle='-.', color=lrho.get_c(), label='$r$')
lf, = ax_f.loglog(dh.Eps_grid, dh.f_grid, label="$f$", color=lrho.get_c())

ldf, = ax_f.loglog(dh.Eps_grid, np.abs(haloModel.dfHalo_dt(R[0], v_cut=v_0)*Tini_orb), 
                                  linestyle='--', color=lrho.get_c(), label="$|\Delta f|$")
lmr = ax_rho.axvline(sp.m1/R[0], linestyle='-.', color=lrho.get_c(), label='$m1/r$')
l = [lrho, lrho_v0, lf, ldf, lr, lmr]


ax_rho.set_ylabel(r'$\rho$ / $pc^{-2}$', fontsize=20); ax_rho.set_xlabel(r'$r$ / $r_{isco}$', fontsize=20); ax_rho.grid()
ax_f.set_ylabel(r'$f$ / $pc^{-2}$',fontsize=20); ax_f.set_xlabel(r'$\epsilon$', fontsize=20); ax_f.grid()
ax_rho.set_xlim((r_grid[0]/sp.r_isco(), r_grid[-1]/sp.r_isco()))
fig.legend(fontsize=20, loc='center right')

def init_plot():
    return l

def update_plot(frame):
    index = frame
    updt = True
    print( index, t[index])
    if updt:
        ax_rho.set_title(f"t_1={t[index]/ms.year_to_pc : .4f} yrs")
        lrho, lrho_v0, lf, ldf, lr, lmr = l
        dh.f_grid = f[index,:]; 
        lrho.set_data(r_grid/sp.r_isco(), dh.density(r_grid))
        lrho_v0.set_data(r_grid/sp.r_isco(), dh.density(r_grid, v_max=[sp.omega_s_approx(r)*r for r in r_grid]))
        lr.set_data( R[index]/sp.r_isco(), lr.get_ydata())
        lf.set_data(dh.Eps_grid, dh.f_grid)
        
        if index < len(t)-1:
            delta_t = t[index+1] - t[index]
            v_0 = sp.omega_s_approx(R[index])*R[index]
            ldf.set_data(dh.Eps_grid, np.abs(haloModel.dfHalo_dt(R[index], v_cut=v_0)*delta_t))

        lmr.set_data(sp.m1/R[index], lmr.get_ydata())
    return l


ani = FuncAnimation(fig, update_plot, frames=n_frame, blit=True, init_func=init_plot , interval=100, repeat=True)
ani.save("HaloFeedbackEvolution." + model + ".mp4")
plt.show()

In [ ]:
# Calculate Gravitational Wave Signal
omega_s = sp.omega_s_approx(R)
f_gw, h, _, __, ___, PhiTild, ____ = waveform.h_2(sp, t, omega_s, R, dbg=True)

In [ ]:
# Compare to model w/o HaloFeedback
sp_2 = ms.SystemProp(m1, m2, NFWSpike, D)
t_2, R_2 = inspiral.Classic.evolve_circular_binary(sp_2, R0, R_fin = R_fin, acc=1e-10)
omega_s_2 = sp_2.omega_s_approx(R_2)
#print(len(t2), t2)
f_gw2, h2, _, __, ___, PhiTild2, ____ = waveform.h_2(sp_2, t_2, omega_s_2, R_2, dbg=True)

In [ ]:
# And to model w/o DM halo
sp_0 = ms.SystemProp(m1, m2, halo.ConstHalo(0.), D)
t_0, R_0 = inspiral.Classic.evolve_circular_binary(sp_0, R0, R_fin = R_fin, acc=1e-10)
omega_s_0 = sp_0.omega_s_approx(R_0)
#print(len(t0), t0)
f_gw0, h0, _, __, ___, PhiTild0, ____ = waveform.h_2(sp_0, t_0, omega_s_0, R_0, dbg=True)
from scipy.interpolate import interp1d
PhiTild0int = interp1d(f_gw0, PhiTild0, kind='cubic', bounds_error=False, fill_value=(0., 0.))

In [ ]:
# Plot phase difference
f_gw5yr = 1./np.pi * (5./256./ (5.*ms.year_to_pc))**(3./8.) * sp_0.m_chirp()**(-5./8.)
plt.figure(figsize=(10,6))
plt.plot(f_gw/ms.hz_to_invpc, np.abs(PhiTild - PhiTild0int(f_gw)), label='$\Delta\Phi_{HF}$')
plt.plot(f_gw2/ms.hz_to_invpc, np.abs(PhiTild2 - PhiTild0int(f_gw2)), label='$\Delta\Phi_{static}$')
plt.axvline(f_gw5yr/ms.hz_to_invpc, linestyle='--')
plt.grid(); plt.legend();
plt.xscale('log'); plt.xlabel('f / Hz'); #plt.xlim(1e-2, 1.); 
plt.yscale('log');   plt.ylabel(r'$\Delta \Phi$'); #plt.ylim(1., 1e8);
plt.savefig("deltaPhi." + model + ".jpg")

In [ ]:
# Plot GW strain
plt.figure(figsize=(10,6))
plt.plot(f_gw0 /ms.hz_to_invpc, 2.*f_gw0*h0 , label='$h_c^0$')
plt.plot(f_gw  /ms.hz_to_invpc, 2.*f_gw*h   , label='$h_c$')
plt.plot(f_gw2 /ms.hz_to_invpc, 2.*f_gw2*h2 , label='$h_c^2$')
plt.axvline(f_gw5yr/ms.hz_to_invpc, linestyle='--')

import imripy.detector as detector
freq = np.geomspace(detector.Lisa().Bandwith()[0], detector.Lisa().Bandwith()[1], 100)
plt.plot(freq/ms.hz_to_invpc, detector.Lisa().NoiseStrain(freq), label='LISA')
plt.grid(); plt.legend();
plt.xscale('log'); plt.xlabel('f / Hz')
plt.yscale('log'); plt.ylabel('characteristic strain')
plt.savefig("strain." + model + ".jpg")